# Data Aggregation and Group Operations

In [1]:
import numpy as np
from numpy.random import randn

import pandas as pd
from pandas import Series, DataFrame

## GroupBy Mechanics

In [2]:
df = DataFrame({
    'key1': ['a','a','b','b','a'],
    'key2': ['A','B','A','B','A'],
    'data1': np.arange(0,5),
    'data2': np.arange(0,10,2)
})
df

,data1,data2,key1,key2
0,0,0,a,A
1,1,2,a,B
2,2,4,b,A
3,3,6,b,B
4,4,8,a,A


In [3]:
# 把 key1 的值当键，对 data1 column 进行分组
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
for key, group in grouped:
    print key
    print group

a
0    0
1    1
4    4
Name: data1, dtype: int64
b
2    2
3    3
Name: data1, dtype: int64


In [5]:
# 求取各组平均值
grouped.mean()

key1
a    1.666667
b    2.500000
Name: data1, dtype: float64

In [6]:
# 把 key1, key2 的值当键，对 data1 column 进行分组求平均值
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     A       2
      B       1
b     A       2
      B       3
Name: data1, dtype: int64

In [7]:
# 如不指定 columns，则对所有数值数据 (data1, data2) 分组
df.groupby('key1').mean()

,data1,data2
key1,,
a,1.666667,3.333333
b,2.500000,5.000000


In [8]:
# 可指定多组键进行分组
df.groupby(['key1', 'key2']).mean()

data1  data2
key1 key2              
a    A         2      4
     B         1      2
b    A         2      4
     B         3      6

### Iterating Over Groups

In [9]:
for name, group in df.groupby('key1'):
    print name
    print group

a
   data1  data2 key1 key2
0      0      0    a    A
1      1      2    a    B
4      4      8    a    A
b
   data1  data2 key1 key2
2      2      4    b    A
3      3      6    b    B


In [10]:
# 除了沿 index 轴，还可以沿 column 轴(axix=1) 进行分组
for type, group in df.groupby(df.dtypes, axis=1):
    print type
    print group

int64
   data1  data2
0      0      0
1      1      2
2      2      4
3      3      6
4      4      8
object
  key1 key2
0    a    A
1    a    B
2    b    A
3    b    B
4    a    A


### Selecting a Column or Subset of Columns

In [11]:
# df['data1'].groupby(df['key1']).count() 的语法糖
df.groupby('key1')['data1'].count()

key1
a    3
b    2
Name: data1, dtype: int64

### Grouping with Dicts and Series

In [12]:
people = DataFrame(
    np.arange(25).reshape((5,5)),
    columns=['a','b','c','d','e'],
    index=['one','two','three','four','five']
)
people

,a,b,c,d,e
one,0,1,2,3,4
two,5,6,7,8,9
three,10,11,12,13,14
four,15,16,17,18,19
five,20,21,22,23,24


In [13]:
# 透过 dict 转换
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
          'd': 'blue', 'e': 'red', 'f': 'orange'}

people.groupby(mapping, axis=1).sum()

,blue,red
one,5,5
two,15,20
three,25,35
four,35,50
five,45,65


In [14]:
# 透过 Series 转换 （index当做键）
map_series = Series(mapping)
people.groupby(map_series, axis=1).sum()

,blue,red
one,5,5
two,15,20
three,25,35
four,35,50
five,45,65


### Grouping with Functions

In [15]:
# 使用函数结果进行分组
people.groupby(len).sum()

,a,b,c,d,e
3,5,7,9,11,13
4,35,37,39,41,43
5,10,11,12,13,14


In [16]:
# 使用匿名函数结果进行分组
people.groupby(lambda x: x[0]).sum()

,a,b,c,d,e
f,35,37,39,41,43
o,0,1,2,3,4
t,15,17,19,21,23


### Grouping by Index Levels

In [17]:
# 创建 column 阶层
columns = pd.MultiIndex.from_arrays(
    [['B','B','B','R','R'], [1,3,5,1,3]], 
    names=['color', 'num'])

hier_df = DataFrame(randn(2,5), columns=columns)
hier_df

color         B                             R          
num           1         3         5         1         3
0      0.119204 -0.183624  0.967715 -0.018133 -0.048289
1      0.120694  1.313170 -1.774178  0.022847 -0.742566

In [18]:
# 根据指定 level 分组
hier_df.groupby(level='color', axis=1).count()

color,B,R
0,3,2
1,3,2


## Data Aggregation

In [19]:
grouped = df.groupby('key1')
for key, group in grouped:
    print key
    print group

a
   data1  data2 key1 key2
0      0      0    a    A
1      1      2    a    B
4      4      8    a    A
b
   data1  data2 key1 key2
2      2      4    b    A
3      3      6    b    B


In [20]:
# 使用 quantile 计算样本分位数
grouped['data1'].quantile(0.5)

key1
a    1.0
b    2.5
Name: data1, dtype: float64

In [21]:
# 使用 agg 传入函数
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,4,8
b,1,2


In [22]:
# 计算分组平均数
grouped['data1'].mean()

key1
a    1.666667
b    2.500000
Name: data1, dtype: float64

In [23]:
# 计算分组中位数
grouped['data1'].median()

key1
a    1.0
b    2.5
Name: data1, dtype: float64

### Column-wise and Multiple Function Application

In [24]:
tips = pd.read_csv('ch08/tips.csv')
tips[:3]

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


In [25]:
# 加入 tip_pct column
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:3]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587


In [26]:
# 按照 sex, smoker 分组
grouped = tips.groupby(['sex','smoker'])

In [27]:
# 针对 tip_pct 数值，分组计算平均
grouped['tip_pct'].agg('mean').unstack()

smoker,No,Yes
sex,,
Female,0.156921,0.182150
Male,0.160669,0.152771


In [28]:
# 针对 tip_pct 数值，分组计算 mean, std, peak_to_peak
grouped['tip_pct'].agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
sex    smoker                                  
Female No      0.156921  0.036421      0.195876
       Yes     0.182150  0.071595      0.360233
Male   No      0.160669  0.041849      0.220186
       Yes     0.152771  0.090588      0.674707

In [29]:
# 变更分组计算的 column name
grouped['tip_pct'].agg([('MEAN', 'mean'), ('STD','std')])

MEAN       STD
sex    smoker                    
Female No      0.156921  0.036421
       Yes     0.182150  0.071595
Male   No      0.160669  0.041849
       Yes     0.152771  0.090588

In [30]:
# 针对不同 column，进行不同的分组计算
grouped.agg({'tip': 'max', 'size': 'sum'})

tip  size
sex    smoker            
Female No       5.2   140
       Yes      6.5    74
Male   No       9.0   263
       Yes     10.0   150

### Returning Aggregated Data in “unindexed” Form

In [31]:
# 把分组键变成 column
tips.groupby(['sex','smoker'], as_index=False).mean()

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771


## Group-wise Operations and Transformations

In [32]:
people.groupby(len).mean()

,a,b,c,d,e
3,2.5,3.5,4.5,5.5,6.5
4,17.5,18.5,19.5,20.5,21.5
5,10.0,11.0,12.0,13.0,14.0


In [33]:
# 将函数应用到各个分组
people.groupby(len).transform('mean')

,a,b,c,d,e
one,2.5,3.5,4.5,5.5,6.5
two,2.5,3.5,4.5,5.5,6.5
three,10.0,11.0,12.0,13.0,14.0
four,17.5,18.5,19.5,20.5,21.5
five,17.5,18.5,19.5,20.5,21.5


In [34]:
# 将函数应用到各个分组
people.groupby(len).transform(lambda g: g - g.mean())

,a,b,c,d,e
one,-2.5,-2.5,-2.5,-2.5,-2.5
two,2.5,2.5,2.5,2.5,2.5
three,0.0,0.0,0.0,0.0,0.0
four,-2.5,-2.5,-2.5,-2.5,-2.5
five,2.5,2.5,2.5,2.5,2.5


### Apply: General split-apply-combine

In [35]:
# 根据 column 找出最高几笔
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [36]:
# 根据 smoker 分组，找出每组最高几笔
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [37]:
# 分组后，舍弃分组键
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis

In [38]:
frame = DataFrame({'data1': randn(1000), 'data2': randn(1000)})

In [39]:
# 根据 data1 的数值，分成四个等长的区间
factor = pd.cut(frame.data1, 4)
factor[:4]

0     (0.235, 1.807]
1    (-1.337, 0.235]
2     (0.235, 1.807]
3     (0.235, 1.807]
Name: data1, dtype: category
Categories (4, object): [(-2.915, -1.337] < (-1.337, 0.235] < (0.235, 1.807] < (1.807, 3.379]]

In [40]:
# 根据（数值）区间分组
grouped = frame['data2'].groupby(factor)
grouped.apply(lambda x: x.describe()).unstack()

,count,mean,std,min,25%,50%,75%,max
data1,,,,,,,,
"(-2.915, -1.337]",86.0,0.089141,1.092366,-2.242967,-0.774336,0.143970,0.738904,2.947698
"(-1.337, 0.235]",499.0,0.048182,1.005677,-2.814036,-0.556956,-0.028885,0.739795,3.174503
"(0.235, 1.807]",380.0,0.086525,1.066747,-2.697566,-0.693880,0.025035,0.738369,3.332718
"(1.807, 3.379]",35.0,-0.303691,1.205436,-3.550983,-1.084854,-0.130039,0.457499,2.056528


In [41]:
# 根据 data1 的样本数，分成四个等长的区间
grouping = pd.qcut(frame.data1, 4, labels=False)
grouping[:4]

0    3
1    0
2    2
3    3
Name: data1, dtype: int64

In [42]:
# 根据（样本数）区间分组
grouped = frame['data2'].groupby(grouping)
grouped.apply(lambda x: x.describe()).unstack()

,count,mean,std,min,25%,50%,75%,max
data1,,,,,,,,
0,250.0,0.049716,1.045626,-2.659531,-0.613507,-0.016968,0.782821,2.947698
1,250.0,0.062784,0.962770,-2.550173,-0.537597,0.033842,0.693180,2.777986
2,250.0,0.088210,1.099001,-2.814036,-0.608631,-0.012498,0.741208,3.332718
3,250.0,0.015126,1.072298,-3.550983,-0.727492,0.027701,0.688088,2.883199


### Example: Filling Missing Values with Group-specific Values

In [43]:
s = Series(randn(6))
s[::2] = np.nan
s

0         NaN
1   -1.455483
2         NaN
3    0.758770
4         NaN
5   -0.654724
dtype: float64

In [44]:
# 使用平均数填充缺失值
s.fillna(s.mean())

0   -0.450479
1   -1.455483
2   -0.450479
3    0.758770
4   -0.450479
5   -0.654724
dtype: float64

In [45]:
chars = ['a','b','c','d','e','f','g','h']
group_key = ['red']*4 + ['blue']*4

data = Series(randn(8), index=chars)
data[['b','d','g']] = np.nan
data

a    0.890841
b         NaN
c   -1.939903
d         NaN
e   -0.505779
f    0.462021
g         NaN
h   -0.404974
dtype: float64

In [46]:
# 使用分组平均数填充缺失值
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

a    0.890841
b   -0.524531
c   -1.939903
d   -0.524531
e   -0.505779
f    0.462021
g   -0.149577
h   -0.404974
dtype: float64

In [47]:
# 使用分组缺省值填充缺失值
fill_values = {'red': -0.5, 'blue': 0.5}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

a    0.890841
b   -0.500000
c   -1.939903
d   -0.500000
e   -0.505779
f    0.462021
g    0.500000
h   -0.404974
dtype: float64

### Example: Random Sampling and Permutation

In [48]:
suits = ['H', 'S', 'C', 'D']
card_val = [1,2,3,4,5,6,7,8,9,10,10,10,10]*4
base_names = ['A'] + range(2,11) + ['J','Q','K']
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = Series(card_val, index=cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
QH     10
KH     10
dtype: int64

In [49]:
# 随机抽取
def draw(deck, n=5):
    return deck.take(np.random.permutation(len(deck))[:n])

draw(deck)

AH     1
AD     1
KH    10
2C     2
9D     9
dtype: int64

In [50]:
# 分组随机抽取
get_suit = lambda card: card[-1]
deck.groupby(get_suit).apply(draw, n=2)

C  3C      3
   5C      5
D  4D      4
   8D      8
H  8H      8
   JH     10
S  10S    10
   5S      5
dtype: int64

### Example: Group Weighted Average and Correlation

In [51]:
df = DataFrame({
    'category': ['a','a','a','a','b','b','b','b'],
    'data': [1]*8,
    'weights': np.random.rand(8)
})

df

,category,data,weights
0,a,1,0.041957
1,a,1,0.569504
2,a,1,0.852348
3,a,1,0.480745
4,b,1,0.382657
5,b,1,0.140593
6,b,1,0.224965
7,b,1,0.643828


In [52]:
# 分组加权平均
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    1.0
b    1.0
dtype: float64

In [53]:
close_px = pd.read_csv('ch09/stock_px.csv',
                       parse_dates=True,
                       index_col=0)
close_px[:4]

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93


In [54]:
rets = close_px.pct_change().dropna()
by_year = rets.groupby(lambda x: x.year)

# 各家股价与 SPX 的相关系数
by_year.apply(lambda x: x.corrwith(x['SPX']))

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [55]:
# AAPL 与 MSFT 的相关系数
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-wise Linear Regression

In [56]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## Pivot Tables and Cross-Tabulation

In [57]:
# 根据 sex, smoker分组，计算平均
tips.pivot_table(index=['sex', 'smoker'])

size       tip   tip_pct  total_bill
sex    smoker                                          
Female No      2.592593  2.773519  0.156921   18.105185
       Yes     2.242424  2.931515  0.182150   17.977879
Male   No      2.711340  3.113402  0.160669   19.791237
       Yes     2.500000  3.051167  0.152771   22.284500

In [58]:
tips.groupby(['sex','smoker']).mean()

total_bill       tip      size   tip_pct
sex    smoker                                          
Female No       18.105185  2.773519  2.592593  0.156921
       Yes      17.977879  2.931515  2.242424  0.182150
Male   No       19.791237  3.113402  2.711340  0.160669
       Yes      22.284500  3.051167  2.500000  0.152771

In [59]:
# 根据 sex, day 分组，计算 tip_pct, size 平均，smoker 放在 column
tips.pivot_table(['tip_pct', 'size'],
                 index=['sex', 'day'],
                 columns='smoker')

tip_pct                size          
smoker             No       Yes        No       Yes
sex    day                                         
Female Fri   0.165296  0.209129  2.500000  2.000000
       Sat   0.147993  0.163817  2.307692  2.200000
       Sun   0.165710  0.237075  3.071429  2.500000
       Thur  0.155971  0.163073  2.480000  2.428571
Male   Fri   0.138005  0.144730  2.000000  2.125000
       Sat   0.162132  0.139067  2.656250  2.629630
       Sun   0.158291  0.173964  2.883721  2.600000
       Thur  0.165706  0.164417  2.500000  2.300000

In [60]:
tips.groupby(['sex', 'day','smoker'])['tip_pct','size'].mean().unstack()

tip_pct                size          
smoker             No       Yes        No       Yes
sex    day                                         
Female Fri   0.165296  0.209129  2.500000  2.000000
       Sat   0.147993  0.163817  2.307692  2.200000
       Sun   0.165710  0.237075  3.071429  2.500000
       Thur  0.155971  0.163073  2.480000  2.428571
Male   Fri   0.138005  0.144730  2.000000  2.125000
       Sat   0.162132  0.139067  2.656250  2.629630
       Sun   0.158291  0.173964  2.883721  2.600000
       Thur  0.165706  0.164417  2.500000  2.300000

In [61]:
# 使用 margins=True 添加分项小计
tips.pivot_table(['tip_pct', 'size'],
                 index=['sex','day'],
                 columns='smoker',
                 margins=True)

tip_pct                          size                    
smoker             No       Yes       All        No       Yes       All
sex    day                                                             
Female Fri   0.165296  0.209129  0.199388  2.500000  2.000000  2.111111
       Sat   0.147993  0.163817  0.156470  2.307692  2.200000  2.250000
       Sun   0.165710  0.237075  0.181569  3.071429  2.500000  2.944444
       Thur  0.155971  0.163073  0.157525  2.480000  2.428571  2.468750
Male   Fri   0.138005  0.144730  0.143385  2.000000  2.125000  2.100000
       Sat   0.162132  0.139067  0.151577  2.656250  2.629630  2.644068
       Sun   0.158291  0.173964  0.162344  2.883721  2.600000  2.810345
       Thur  0.165706  0.164417  0.165276  2.500000  2.300000  2.433333
All          0.159328  0.163196  0.160803  2.668874  2.408602  2.569672

In [62]:
# 使用 aggfunc 指定其他聚合函数
tips.pivot_table('tip_pct', 
                 index=['sex','smoker'],
                 columns='day',
                 aggfunc=len,
                 margins=True)

day             Fri   Sat   Sun  Thur    All
sex    smoker                               
Female No       2.0  13.0  14.0  25.0   54.0
       Yes      7.0  15.0   4.0   7.0   33.0
Male   No       2.0  32.0  43.0  20.0   97.0
       Yes      8.0  27.0  15.0  10.0   60.0
All            19.0  87.0  76.0  62.0  244.0

In [63]:
# 使用 fill_value 填充缺失值
tips.pivot_table('size',
                index=['time','sex','smoker'],
                columns='day',
                aggfunc='sum',
                fill_value=0)

day                   Fri  Sat  Sun  Thur
time   sex    smoker                     
Dinner Female No        2   30   43     2
              Yes       8   33   10     0
       Male   No        4   85  124     0
              Yes      12   71   39     0
Lunch  Female No        3    0    0    60
              Yes       6    0    0    17
       Male   No        0    0    0    50
              Yes       5    0    0    23

### Cross-Tabulations: Crosstab

In [64]:
data = DataFrame({
    'Sample': [1,2,3,4,5,6,7,8,9,10],
    'Gender': ['F','M','F','M','M','M','F','F','M','F'],
    'Handedness': ['R','L','R','R','L','R','R','L','R','R']
})

data

,Gender,Handedness,Sample
0,F,R,1
1,M,L,2
2,F,R,3
3,M,R,4
4,M,L,5
5,M,R,6
6,F,R,7
7,F,L,8
8,M,R,9
9,F,R,10


In [65]:
# 计算分组频率
pd.crosstab(data.Gender,
            data.Handedness,
            margins=True)

Handedness,L,R,All
Gender,,,
F,1,4,5
M,2,3,5
All,3,7,10


In [66]:
data.groupby(['Gender','Handedness']).apply(len).unstack()

Handedness,L,R
Gender,,
F,1,4
M,2,3


In [67]:
# crosstab 前两个参数可以是数组、Series、数组列表
pd.crosstab([tips.time, tips.day],
           tips.smoker,
           margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

In [68]:
tips.groupby(['time', 'day', 'smoker']).apply(len).unstack().fillna(0)

smoker         No   Yes
time   day             
Dinner Fri    3.0   9.0
       Sat   45.0  42.0
       Sun   57.0  19.0
       Thur   1.0   0.0
Lunch  Fri    1.0   6.0
       Thur  44.0  17.0

## Example: 2012 Federal Election Commission Database

In [69]:
fec = pd.read_csv('ch09/P00000001-ALL.csv', low_memory=False)

In [70]:
fec.ix[123456]

cmte_id                             C00431445
cand_id                             P80003338
cand_nm                         Obama, Barack
contbr_nm                         ELLMAN, IRA
contbr_city                             TEMPE
contbr_st                                  AZ
contbr_zip                          852816719
contbr_employer      ARIZONA STATE UNIVERSITY
contbr_occupation                   PROFESSOR
contb_receipt_amt                          50
contb_receipt_dt                    01-DEC-11
receipt_desc                              NaN
memo_cd                                   NaN
memo_text                                 NaN
form_tp                                 SA17A
file_num                               772372
Name: 123456, dtype: object